In [39]:
import os
# Move working directory 1 level up
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))
print("New working directory:", os.getcwd())

!pip install -r requirements.txt --quiet
print("\n> If you alreadd run ETL file, or any other file that already use `requirement.txt`, \n"
      "please ignore the ERROR message as follow, unless any `import` function get error later, which rarely can happen! \n")

New working directory: g:\

> If you alreadd run ETL file, or any other file that already use `requirement.txt`, 
please ignore the ERROR message as follow, unless any `import` function get error later, which rarely can happen! 



ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [25]:
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

# Load environment variables from .env file, remember to create this file with your credentials
# Read readme file for more information ! 
load_dotenv("postgressql_secretkeys.env")  
# Extract variables
username = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host     = os.getenv("PG_HOST")
port     = os.getenv("PG_PORT")
dbname   = os.getenv("PG_DB")

# Create the SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}")

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    print(result.fetchone())
    print("======================================= \n || -- Connected to PostgreSQL! -- || \n=======================================")


('PostgreSQL 17.4 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit',)
 || -- Connected to PostgreSQL! -- || 


In [26]:
import pandas as pd
# Query a full table with all dimensions and fact from data warehouse query for data mining
query = """
SELECT * 
FROM fact_fatality, dim_crash_group, dim_fatality_demographic, dim_time, dim_location
WHERE fact_fatality.crashgroup_sk = dim_crash_group.crashgroup_sk
AND fact_fatality.fatality_demographics_sk = dim_fatality_demographic.fatality_demographics_sk
AND fact_fatality.time_sk = dim_time.time_sk
AND fact_fatality.state_id = dim_location.state_id
"""
data = pd.read_sql(text(query), con=engine)
data.head(10)

,crash_sk,crashgroup_sk,fatality_demographics_sk,time_sk,state_id,crashgroup_sk,crash_type,speed_limit,road_type,bus_involvement,...,gender,age_group,time_sk,time,dayweek,month,year,state_id,state_name,population
0,1,1,1,1,NSW,1,Single,100,Arterial Road,No,...,Male,65_to_74,1,04,Friday,12,2024,NSW,New South Wales,8342285
1,2,2,2,2,NSW,2,Single,80,Local Road,No,...,Female,17_to_25,2,06,Friday,12,2024,NSW,New South Wales,8342285
2,3,3,3,3,TAS,3,Multiple,50,Local Road,No,...,Female,26_to_39,3,09,Friday,12,2024,TAS,Tasmania,573156
3,4,4,3,4,NSW,4,Multiple,100,National or State Highway,No,...,Female,26_to_39,4,10,Friday,12,2024,NSW,New South Wales,8342285
4,5,5,4,5,VIC,5,Multiple,Unknown,Undetermined,Unknown,...,Male,40_to_64,5,11,Friday,12,2024,VIC,Victoria,6815441
5,6,4,5,6,QLD,4,Multiple,100,National or State Highway,No,...,Female,40_to_64,6,13,Friday,12,2024,QLD,Queensland,5460420
6,7,6,6,6,SA,6,Single,100,Sub-arterial Road,No,...,Male,40_to_64,6,13,Friday,12,2024,SA,South Australia,1852284
7,8,7,1,7,WA,7,Single,90,Undetermined,No,...,Male,65_to_74,7,17,Friday,12,2024,WA,Western Australia,2881227
8,9,8,7,8,QLD,8,Multiple,60,Local Road,No,...,Female,26_to_39,8,19,Friday,12,2024,QLD,Queensland,5460420
9,10,3,8,8,TAS,3,Multiple,50,Local Road,No,...,Male,17_to_25,8,19,Friday,12,2024,TAS,Tasmania,573156


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56874 entries, 0 to 56873
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   crash_sk                       56874 non-null  int64 
 1   crashgroup_sk                  56874 non-null  int64 
 2   fatality_demographics_sk       56874 non-null  int64 
 3   time_sk                        56874 non-null  int64 
 4   state_id                       56874 non-null  object
 5   crashgroup_sk                  56874 non-null  int64 
 6   crash_type                     56874 non-null  object
 7   speed_limit                    56874 non-null  object
 8   road_type                      56874 non-null  object
 9   bus_involvement                56874 non-null  object
 10  heavy_rigid_truck_involvement  56874 non-null  object
 11  articulated_truck_involvement  56874 non-null  object
 12  fatality_demographics_sk       56874 non-null  int64 
 13  r

In [28]:
#Check missing values in the dataframe
data.isna()

#Count total missing values at each column in the dataframe
data.isna().sum()

crash_sk                          0
crashgroup_sk                     0
fatality_demographics_sk          0
time_sk                           0
state_id                          0
crashgroup_sk                     0
crash_type                        0
speed_limit                       0
road_type                         0
bus_involvement                   0
heavy_rigid_truck_involvement     0
articulated_truck_involvement     0
fatality_demographics_sk          0
road_user                         0
gender                            0
age_group                         0
time_sk                           0
time                             43
dayweek                           0
month                             0
year                              0
state_id                          0
state_name                        0
population                        0
dtype: int64

In [29]:
# Drop all the columns that are not needed for the analysis
data.drop(columns=['crashgroup_sk', 'fatality_demographics_sk', 'time_sk', 'state_id', "crash_sk", ], inplace=True)
data_2=data.copy()

In [30]:
# Tranform the data to be used in the mining process using apriori algorithm
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

#Data transformation to str and then to list
#Convert all boolean values to column names and values Yes or No
data = data.astype(str)
data = data.apply(lambda row: [f"{col}={val}" for col, val in row.items()], axis=1)

#TransactionEncoder() was designed to covert lists to array
list_ = data.values.tolist()

#Covert the list to one-hot encoded boolean numpy array. 
#Apriori function allows boolean data type only, such as 1 and 0, or FALSE and TRUE.
te = TransactionEncoder()
array_te = te.fit(list_).transform(list_)

#Check the array
array_te

#Check the colunms
te.columns_

#Apriori function can handle dataframe only, covert the array to a dataframe
arm_df = pd.DataFrame(array_te, columns = te.columns_)
arm_df

,age_group=0_to_16,age_group=17_to_25,age_group=26_to_39,age_group=40_to_64,age_group=65_to_74,age_group=75_or_older,age_group=Unknown,articulated_truck_involvement=No,articulated_truck_involvement=Unknown,articulated_truck_involvement=Yes,...,year=2015,year=2016,year=2017,year=2018,year=2019,year=2020,year=2021,year=2022,year=2023,year=2024
0,False,False,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
1,False,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
2,False,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56869,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
56870,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
56871,False,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
56872,True,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [31]:
frequent_itemsets = apriori(arm_df,min_support=0.2,use_colnames =True, )
frequent_itemsets

,support,itemsets
0,0.255688,(age_group=17_to_25)
1,0.232778,(age_group=26_to_39)
2,0.257446,(age_group=40_to_64)
3,0.899356,(articulated_truck_involvement=No)
4,0.980501,(bus_involvement=No)
...,...,...
212,0.233481,"(articulated_truck_involvement=No, bus_involve..."
213,0.272532,"(articulated_truck_involvement=No, bus_involve..."
214,0.221947,"(heavy_rigid_truck_involvement=Unknown, articu..."
215,0.228681,"(road_user=Driver, articulated_truck_involveme..."


In [32]:
#Find the frequent itemsets
frequent_itemsets = apriori(arm_df,min_support=0.05,use_colnames =True)

#Check the length of rules
frequent_itemsets['length']=frequent_itemsets['itemsets'].apply(lambda x: len(x))

# We use length to filter the rules, we want to find the rules with length >= 2 so it wont be only one item in the rule
# We also filter the rules with support >= 0.3, so we can find the most frequent itemsets

length = 2 # Min Length of the patterns we want to find
support = 0.3 # Min Support of the patterns we want to find
frequent_itemsets[ (frequent_itemsets['length']>=1) & 
                  (frequent_itemsets['support']>=0.1)]

KeyboardInterrupt: 

In [ ]:
#Assume the min confidence is 0.5
rules_con = association_rules(frequent_itemsets, metric="confidence",min_threshold=0.5)
rules_con[["antecedents","consequents", "antecedent support", "consequent support", "support", "confidence", "lift"]]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift
0,(age_group=0_to_16),(articulated_truck_involvement=No),0.075236,0.899356,0.069434,0.922879,1.026155
1,(age_group=0_to_16),(bus_involvement=No),0.075236,0.980501,0.073566,0.977799,0.997244
2,(age_group=0_to_16),(road_type=Undetermined),0.075236,0.811197,0.065900,0.875906,1.079770
3,(age_group=17_to_25),(articulated_truck_involvement=No),0.255688,0.899356,0.236136,0.923532,1.026881
4,(age_group=17_to_25),(bus_involvement=No),0.255688,0.980501,0.252066,0.985834,1.005439
...,...,...,...,...,...,...,...
24463,"(gender=Male, road_user=Driver, population=681...","(state_name=Victoria, articulated_truck_involv...",0.074234,0.155537,0.050744,0.683562,4.394859
24464,"(road_type=Undetermined, road_user=Driver, pop...","(state_name=Victoria, articulated_truck_involv...",0.082234,0.132574,0.050744,0.617062,4.654482
24465,"(heavy_rigid_truck_involvement=No, road_user=D...","(state_name=Victoria, articulated_truck_involv...",0.095949,0.112477,0.050744,0.528862,4.701969
24466,"(road_user=Driver, state_name=Victoria)","(articulated_truck_involvement=No, bus_involve...",0.100186,0.109171,0.050744,0.506494,4.639445


In [ ]:
#Assume the min lift is 1
rules_lift = association_rules(frequent_itemsets, metric="lift",min_threshold=1)

#output antecedents, consequents, support, confidence and lift.
result_arm = rules_con[['antecedents','consequents','support','confidence','lift']]
result_arm

,antecedents,consequents,support,confidence,lift
0,(age_group=0_to_16),(articulated_truck_involvement=No),0.069434,0.922879,1.026155
1,(age_group=0_to_16),(bus_involvement=No),0.073566,0.977799,0.997244
2,(age_group=0_to_16),(road_type=Undetermined),0.065900,0.875906,1.079770
3,(age_group=17_to_25),(articulated_truck_involvement=No),0.236136,0.923532,1.026881
4,(age_group=17_to_25),(bus_involvement=No),0.252066,0.985834,1.005439
...,...,...,...,...,...
24463,"(gender=Male, road_user=Driver, population=681...","(state_name=Victoria, articulated_truck_involv...",0.050744,0.683562,4.394859
24464,"(road_type=Undetermined, road_user=Driver, pop...","(state_name=Victoria, articulated_truck_involv...",0.050744,0.617062,4.654482
24465,"(heavy_rigid_truck_involvement=No, road_user=D...","(state_name=Victoria, articulated_truck_involv...",0.050744,0.528862,4.701969
24466,"(road_user=Driver, state_name=Victoria)","(articulated_truck_involvement=No, bus_involve...",0.050744,0.506494,4.639445


In [ ]:
#Find the rules whose confidence >= 0.7
new_result_arm = result_arm[result_arm['confidence']>=0.5]
new_result_arm

,antecedents,consequents,support,confidence,lift
0,(age_group=0_to_16),(articulated_truck_involvement=No),0.069434,0.922879,1.026155
1,(age_group=0_to_16),(bus_involvement=No),0.073566,0.977799,0.997244
2,(age_group=0_to_16),(road_type=Undetermined),0.065900,0.875906,1.079770
3,(age_group=17_to_25),(articulated_truck_involvement=No),0.236136,0.923532,1.026881
4,(age_group=17_to_25),(bus_involvement=No),0.252066,0.985834,1.005439
...,...,...,...,...,...
24463,"(gender=Male, road_user=Driver, population=681...","(state_name=Victoria, articulated_truck_involv...",0.050744,0.683562,4.394859
24464,"(road_type=Undetermined, road_user=Driver, pop...","(state_name=Victoria, articulated_truck_involv...",0.050744,0.617062,4.654482
24465,"(heavy_rigid_truck_involvement=No, road_user=D...","(state_name=Victoria, articulated_truck_involv...",0.050744,0.528862,4.701969
24466,"(road_user=Driver, state_name=Victoria)","(articulated_truck_involvement=No, bus_involve...",0.050744,0.506494,4.639445


In [ ]:
# Filter the new_result_arm by containing "road_user"
filtered_rules = new_result_arm[
    new_result_arm['consequents'].astype(str).str.contains('road_user=', case=False)
]
# Sort the filtered rules by lift and confidence
filtered_sorted_rules = filtered_rules.sort_values(by=['confidence','lift',], ascending=False, inplace=False)
# With k = 5, we can find the top 5 rules
k_5_result = filtered_sorted_rules[:5]
filtered_sorted_rules

,antecedents,consequents,support,confidence,lift
5922,"(gender=Male, articulated_truck_involvement=Ye...",(road_user=Driver),0.050093,0.682232,1.510893
1296,"(gender=Male, articulated_truck_involvement=Yes)",(road_user=Driver),0.050515,0.680644,1.507378
5924,"(gender=Male, articulated_truck_involvement=Yes)","(road_user=Driver, bus_involvement=No)",0.050093,0.674959,1.512990
3022,"(age_group=26_to_39, speed_limit=100, bus_invo...",(road_user=Driver),0.052010,0.633948,1.403962
497,"(age_group=26_to_39, speed_limit=100)",(road_user=Driver),0.052397,0.630288,1.395856
...,...,...,...,...,...
13274,"(gender=Male, heavy_rigid_truck_involvement=No...","(road_user=Driver, articulated_truck_involveme...",0.077100,0.500742,1.287604
2994,"(heavy_rigid_truck_involvement=No, age_group=2...","(road_user=Driver, bus_involvement=No)",0.069997,0.500503,1.121930
20946,"(heavy_rigid_truck_involvement=Unknown, road_t...","(road_user=Driver, bus_involvement=No)",0.053328,0.500495,1.121912
9425,"(gender=Male, heavy_rigid_truck_involvement=No...",(road_user=Driver),0.060748,0.500072,1.107477


In [ ]:
filtered_sorted_rules

,antecedents,consequents,support,confidence,lift
5922,"(gender=Male, articulated_truck_involvement=Ye...",(road_user=Driver),0.050093,0.682232,1.510893
1296,"(gender=Male, articulated_truck_involvement=Yes)",(road_user=Driver),0.050515,0.680644,1.507378
5924,"(gender=Male, articulated_truck_involvement=Yes)","(road_user=Driver, bus_involvement=No)",0.050093,0.674959,1.512990
3022,"(age_group=26_to_39, speed_limit=100, bus_invo...",(road_user=Driver),0.052010,0.633948,1.403962
497,"(age_group=26_to_39, speed_limit=100)",(road_user=Driver),0.052397,0.630288,1.395856
...,...,...,...,...,...
13274,"(gender=Male, heavy_rigid_truck_involvement=No...","(road_user=Driver, articulated_truck_involveme...",0.077100,0.500742,1.287604
2994,"(heavy_rigid_truck_involvement=No, age_group=2...","(road_user=Driver, bus_involvement=No)",0.069997,0.500503,1.121930
20946,"(heavy_rigid_truck_involvement=Unknown, road_t...","(road_user=Driver, bus_involvement=No)",0.053328,0.500495,1.121912
9425,"(gender=Male, heavy_rigid_truck_involvement=No...",(road_user=Driver),0.060748,0.500072,1.107477


Confidence — how often rule is correct
confidence(A ⇒ B) = P(B | A) = P(A ∩ B) / P(A)

Lift — how much more likely B is given A
lift(A ⇒ B) = P(B | A) / P(B) = confidence / support(B)

Therefore I will put confidence first, lift later (Already done)

Big question: How to explain this

And references for all of this

In [ ]:
# This is just a melody to play when the script is finished
# This to let we know that the script is finished and we can do something else while waiting
# Remove if dont need it

import winsound
import time

# Short funny retro-style melody
notes = [
    (659, 150),  # E5
    (784, 150),  # G5
    (880, 150),  # A5
    (659, 300),  # E5
    (523, 300),  # C5
    (587, 300),  # D5
    (659, 600),  # E5 long
]
for freq, dur in notes:
    winsound.Beep(freq, dur)
    time.sleep(0.05)  
for freq, dur in notes:
    winsound.Beep(freq, dur)
    time.sleep(0.05)  
